<a href="https://colab.research.google.com/github/enesemretas/mcpath-colab/blob/main/notebooks/mcpath_form.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title MCPath (User Interface Only)
# This cell only loads the UI from the GitHub module.
!pip -q install pyyaml ipywidgets requests

import sys, subprocess, pathlib
REPO = "https://github.com/enesemretas/mcpath-colab.git"

# Clone once if not already present
if not pathlib.Path("mcpath-colab").exists():
    subprocess.run(["git", "clone", "--depth", "1", REPO], check=True)

sys.path.append("/content/mcpath-colab")   # so we can import the module
from mcpath.ui import launch

launch(
    defaults_url="https://raw.githubusercontent.com/enesemretas/mcpath-colab/main/config/defaults.yaml",
    show_title="MCPath-style Parameters"
)


In [2]:
# @title
# ▶️ One-click launcher for the MCPath form

import importlib, sys
from IPython.display import display, clear_output
import ipywidgets as W

# If you're in Colab, this helps widgets render properly
try:
    from google.colab import output as _colab_output
    _colab_output.enable_custom_widget_manager()
except Exception:
    pass

# Where your package lives (adjust if needed)
PKG_ROOT = "/content/mcpath-colab"
if PKG_ROOT not in sys.path:
    sys.path.append(PKG_ROOT)

# UI elements
start_btn = W.Button(
    description="Start MCPath Form",
    icon="play",
    button_style="success",
    layout=W.Layout(width="240px", height="42px")
)
status_lbl = W.Label(value="")
holder = W.Output()   # the form will render here

def _launch_form(_btn=None):
    status_lbl.value = "Launching form…"
    with holder:
        clear_output()
        try:
            # Import (or reload) and launch
            import mcpath.ui as mcui
            importlib.reload(mcui)
            mcui.launch(
                defaults_url="https://raw.githubusercontent.com/enesemretas/mcpath-colab/main/config/defaults.yaml",
                show_title="MCPath (Python readpdb)"
            )
            status_lbl.value = "Form ready."
        except Exception as e:
            status_lbl.value = f"Error: {e}"

start_btn.on_click(_launch_form)

# Display the launcher UI
display(W.HBox([start_btn, status_lbl]))
display(holder)


Output()